## A more generalised form of attention

There are some issues with the method described in bahdanau. First, it's hard-tied to a RNN based encoder decoder model. What if you don't want to use recurrent networks. Second, it needs the outputs from the encoder and decoder to form the information space (encoder state) and the search space (decoder output) to get probable output candidates; but you may not always want to model seq2seq using attention. 


So there is a more generalised version of attention, which uses the terms key, query and values to describe the attention process. 

To find parallels with the bahdanau method, 

query : what we want to know or search (decoder output)

key: what information we have (and can be used for searching) , (the encoder state)

value: the probability of the search result being related to the query (in the bahdanau paper, values are the same as the keys)



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from einops import rearrange
import numpy as np

So the implementation details become this:

Say you have an input sequence $X$,

Then you need a query representation (with a query weight)
A key rep (same, another weight) and so on for value

In [2]:
# size for the input
seq_len = 50
x = torch.arange(seq_len)

In [3]:
# okay this here becomes more of a self-attention
# so when both query and key come from the input sequence,
# attention is comparing the input to itself
# becomes self attetnion

@torch.no_grad()
def attention(sequence: torch.Tensor) -> torch.Tensor:
    # modules
    embed = nn.Embedding(100, 10)
    Q = nn.Linear(10, 10)
    K = nn.Linear(10, 10)
    V = nn.Linear(10, 10)
    
    x = embed(sequence)
    q = Q(x)
    k = K(x)
    v = V(x)
    
    # now that we have q, k, v
    
    attention_scores = q @ k.T
    alpha = attention_scores.softmax(dim=-1)
    # attention
    a = attention_scores @ q
    return a


a = attention(x)  

In [4]:
a.size()

torch.Size([50, 10])

In [5]:
a

tensor([[-1.6716e+01,  1.1041e+01, -1.1626e+01,  1.7849e+01, -1.7858e+01,
          5.4379e+00, -1.6980e+01,  2.4383e+01,  5.2313e+00, -1.3040e+01],
        [-4.5440e+00,  2.7986e+01,  3.2808e+00, -1.9986e+00, -2.3787e+01,
          1.1641e+00,  1.6745e+01,  1.0561e+01,  6.5427e+00, -1.0154e+01],
        [ 1.2910e+01, -2.3780e+01,  6.0605e+00, -1.2543e+01,  1.2747e+01,
         -2.4295e+00,  3.5230e+00, -1.7888e+01, -2.0508e+00,  1.1265e+01],
        [ 7.4843e+00, -6.9338e+00, -5.6274e+00,  3.3967e+00,  6.3810e+00,
          6.5698e+00, -1.1233e+00, -4.4610e+00, -3.9812e+00,  4.2825e+00],
        [-3.4243e+00,  2.6711e+00, -2.1215e-01, -2.2021e+00,  4.1969e-01,
          1.1941e+00, -4.6860e+00,  3.1345e+00,  7.6051e+00, -1.4732e+01],
        [ 2.4758e+00,  2.3400e+00,  3.3501e+00,  1.3557e+01,  9.1841e+00,
          6.3338e+00, -1.7144e+01,  1.1330e+01, -1.3698e+00, -2.7343e+01],
        [ 1.3977e+01, -1.7247e+01,  6.7863e+00, -9.2467e+00,  5.6533e+00,
          8.0504e-01,  1.3043e+0